Case Study: Refactoring Strategy

Strategy is a good example of a design pattern that can be simpler in Python
if you leverage functions as first-class objects.

Example 10-1. Implementation of the Order class with pluggable discount
strategies.